# Contingency Tables

#### Connect to the Database

In [1]:
import psycopg2 as pg
import pandas as pd

# Database setup
host = "localhost"
database = "cdm"
user = "postgres"
password = %env PGPASSWORD
connection_string = "host={} dbname={} user={} password={}".format(host, database, user, password)

db = pg.connect(connection_string)

### Privacy Functions

In [125]:
import numpy as np
import scipy.spatial as sp
def perturb_gender_decade(table, epsilon=0.1):
    for i, row in table.iterrows():
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(row['count'] + noise)
        if cur_count < 0:
            cur_count = 0
        table.set_value(i, 'count', cur_count)
    return table

def perturb_gender_drug(table, epsilon=0.1):
    for i, row in table.iterrows():
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(row['total'] + noise)
        if cur_count < 0:
            cur_count = 0
        table.set_value(i, 'total', cur_count)
    return table

def perturb_length_stay(table, epsilon=0.1):
    for i, row in table.iterrows():
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(row['Number of Patients'] + noise)
        if cur_count < 0:
            cur_count = 0
        table.set_value(i, 'Number of Patients', cur_count)
    return table

def get_latex(df):
    return df.to_latex()
    

## GROUP BY queries

### Heart Failure by Gender and Birth Decade

In [141]:
query = """
    SELECT
        extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
        CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
        COUNT(*)
    FROM person p, condition_occurrence con_oc, concept con
    WHERE p.person_id = con_oc.person_id 
        AND con_oc.condition_concept_id = con.concept_id
        AND (con.concept_name LIKE '%Heart Failure%' OR
            con.concept_name LIKE '%heart failure%')
        AND con.domain_id = 'Condition'
        AND con.concept_class_id='Clinical Finding'
    GROUP BY
        date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
        p.gender_concept_id
    ORDER BY
        birth_decade,
        gender;"""
results = pd.read_sql(query, con=db)
results_perturb = pd.read_sql(query, con=db)

perturb_gender_decade(results_perturb)

df = results.pivot(index='gender', columns='birth_decade', values='count')

df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

hf = df.as_matrix()
hf_p = df_perturb.as_matrix()

cos = 1 - sp.distance.cdist(hf, hf_p, 'cosine')
cos[0][0]

0.98673989065761669

### HIV by Gender and Birth Decade

In [154]:
query = """
SELECT
  extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
  CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
  COUNT(*)
FROM
  condition_occurrence co,
  person p
WHERE
  co.person_id = p.person_id AND
  condition_concept_id = '4241530'
GROUP BY 
  date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
  p.gender_concept_id
ORDER BY
  birth_decade,
  gender;"""
results = pd.read_sql(query, con=db)
results_perturb = pd.read_sql(query, con=db)

perturb_gender_decade(results_perturb)

df = results.pivot(index='gender', columns='birth_decade', values='count')
#df_perturb = results_perturb.pivot(index='gender', columns='birth_decade', values='count')
df_perturb = results_perturb.pivot(index='birth_decade', columns='gender', values='count')

df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

#df_perturb.columns.names = ['Birth Decade']
#df_perturb.index.names = ['Gender']

df_perturb.columns.names = ['Gender']
df_perturb.index.names = ['Birth Decade']

HIV = df.as_matrix()
HIV_p = df_perturb.as_matrix()

#cos = 1 - sp.distance.cdist(HIV, HIV_p, 'cosine')

#print(cos[0][0])
print(df_perturb.to_latex())

\begin{tabular}{lrr}
\toprule
Gender &  Female &   Male \\
Birth Decade &         &        \\
\midrule
1900.0       &    31.0 &    NaN \\
1910.0       &   152.0 &   68.0 \\
1920.0       &   468.0 &  263.0 \\
1930.0       &   602.0 &  468.0 \\
1940.0       &   374.0 &  260.0 \\
1950.0       &   112.0 &  160.0 \\
1960.0       &   156.0 &   95.0 \\
1970.0       &    34.0 &   61.0 \\
1980.0       &    17.0 &   19.0 \\
\bottomrule
\end{tabular}



### Diabetes by Gender and Age

In [143]:
query = """
    SELECT
        extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
        CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
        COUNT(*)
    FROM person p, condition_occurrence con_oc, concept con
    WHERE p.person_id = con_oc.person_id 
        AND con_oc.condition_concept_id = con.concept_id
        AND con.concept_class_id='Clinical Finding'
        AND con_oc.condition_concept_id=con.concept_id
        AND (con.concept_name LIKE '%Diabetes%' OR
            con.concept_name LIKE '%diabetes%')
        AND con.domain_id = 'Condition'
    GROUP BY
        date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
        p.gender_concept_id
    ORDER BY
        birth_decade,
        gender;"""
results = pd.read_sql(query, con=db)

results_perturb = results
results_perturb = perturb_gender_decade(results_perturb)

df = results.pivot(index='gender', columns='birth_decade', values='count')
df_perturb = results_perturb.pivot(index='gender', columns='birth_decade', values='count')

df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

df_perturb.columns.names = ['Birth Decade']
df_perturb.index.names = ['Gender']

diabetes = df.as_matrix()
diabetes_p = df_perturb.as_matrix()

cos = 1 - sp.distance.cdist(diabetes, diabetes_p, 'cosine')
print (cos[0][0])
df_perturb

1.0


Birth Decade,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0
Gender,,,,,,,,,
Female,7454,83010,259309,336060,181375,50155,29579,14738,4255
Male,2302,32384,163313,260815,154870,48177,29422,13845,4001


### Drugs by Gender

In [101]:
query = """
SELECT
  c.concept_name,
  CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
  count(*) AS total
FROM
  person p,
  drug_exposure de,
  top_drugs td,
  concept c
WHERE
  c.concept_id = de.drug_concept_id AND
  de.drug_concept_id = td.concept_id AND
  de.person_id = p.person_id
GROUP BY
  c.concept_name,
  gender
ORDER BY 
  c.concept_name,
  gender;
"""

results = pd.read_sql(query, con=db)

df = results.pivot(index='gender', columns='concept_name', values='total')

df.columns.names = ['Drug Name']
df.index.names = ['Gender']

drugs = df
drugs

Drug Name,Epoetin Alfa,Gemfibrozil 600 MG Oral Tablet,"Influenza virus vaccine, trivalent (IIV3), split virus, 0.5 mL dosage, for intramuscular use",Lovastatin 20 MG Oral Tablet,Omeprazole 20 MG Delayed Release Oral Capsule,Oxygen 99 % Gas for Inhalation,Simvastatin 40 MG Oral Tablet,paricalcitol Injectable Solution
Gender,,,,,,,,
Female,83344,22417,43881,19364,22380,37516,17637,45392
Male,55902,15008,33530,12676,13868,23943,11772,30355


In [115]:
query = """
SELECT 
  sc.count AS stay_length,
  count(*) AS num_patients
FROM
  stay_count sc
GROUP BY
  stay_length
ORDER BY
  stay_length;
"""

results = pd.read_sql(query, con=db)
results = results.set_index('stay_length')

results.columns = ['Number of Patients']
results.index.names = ['Stay Length']
results = perturb_length_stay(results)
results

,Number of Patients
Stay Length,
1,3552
2,3087
3,2689
4,2086
5,1686
6,1297
7,1051
8,814
9,763
